## Обзор данных

In [1]:
import pandas as pd # импорт библиотеки pandas

In [3]:
df.head() # получение первых 5 строк таблицы df

,userID,Track,artist,genre,City,time,Day
0,FFB692EC,Kamigata To Boots,The Mass Missile,rock,Saint-Petersburg,20:28:33,Wednesday
1,55204538,Delayed Because of Accident,Andreas Rönnberg,rock,Moscow,14:07:09,Friday
2,20EC38,Funiculì funiculà,Mario Lanza,pop,Saint-Petersburg,20:58:07,Wednesday
3,A3DD03C9,Dragons in the Sunset,Fire + Ice,folk,Saint-Petersburg,08:37:09,Monday
4,E2DC1FAE,Soul People,Space Echo,dance,Moscow,08:34:34,Monday


In [4]:
df.info() # получение общей информации о данных в таблице df

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 65079 entries, 0 to 65078
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0     userID  65079 non-null  object
 1   Track     63848 non-null  object
 2   artist    57876 non-null  object
 3   genre     63881 non-null  object
 4     City    65079 non-null  object
 5   time      65079 non-null  object
 6   Day       65079 non-null  object
dtypes: object(7)
memory usage: 3.5+ MB


Структура файла:
* `userID` — идентификатор пользователя;
* `Track` — название трека;  
* `artist` — имя исполнителя;
* `genre` — жанр;
* `City` — город;
* `time` — время начала прослушивания;
* `Day` — день недели.

Тип данных во всех слолбцах — `object`.

В названиях колонок видны нарушения стиля:

1. Строчные буквы сочетаются с прописными.
2. Встречаются пробелы.
3. Из названия столбца 'time' не понятно, о каком именно времени идет речь.
4. В названии колонки 'userID' пропущен пробел, не спользован змеиный регистр.

Количество значений в столбцах различается. Значит, в данных есть пропущенные значения.


**Выводы**

Строки содержат информацию о прослушивании трека (характеристики трека и ползователя, его прослушавшего).

Предварительно можно утверждать, что, данных достаточно для проверки гипотез. Но встречаются пропуски в данных, а в названиях колонок — расхождения с хорошим стилем. 

Необходимо выполнить предобработку данных. 

## Предобработка данных

### Стиль заголовков

In [5]:
 df.columns # названия столбцов 

Index(['  userID', 'Track', 'artist', 'genre', '  City  ', 'time', 'Day'], dtype='object')

Приведем названия в соответствие с хорошим стилем

In [6]:
df = df.rename(columns={'  userID':'user_id', 'Track': 'track', '  City  ':'city', 'time': 'time_start', 'Day': 'day'}) # переименование столбцов

In [7]:
df.columns # проверка

Index(['user_id', 'track', 'artist', 'genre', 'city', 'time_start', 'day'], dtype='object')

### Пропущенные значения

In [8]:
df.isna().sum() # подсчёт пропусков

user_id          0
track         1231
artist        7203
genre         1198
city             0
time_start       0
day              0
dtype: int64

In [9]:
#заменим пропущенные значения в столбцах track, artist и genre на строку 'unknown', 
#пропуски в track и artist не влияют на исследование, а причину продусков в genre установить не возможно в рамках проекта
columns_to_replace = ['track', 'artist', 'genre']
for column in columns_to_replace:
    df[column] =  df[column].fillna('unknown')
# перебор названий столбцов в цикле и замена пропущенных значений на 'unknown'

In [10]:
df.isna().sum() # подсчёт пропусков после

user_id       0
track         0
artist        0
genre         0
city          0
time_start    0
day           0
dtype: int64

### Дубликаты

In [11]:
df.duplicated().sum() # подсчёт явных дубликатов

3826

In [12]:
df = df.drop_duplicates().reset_index(drop = True) # удаление явных дубликатов (с удалением старых индексов и формированием новых)

In [13]:
df.duplicated().sum() # проверка 

0

Избавимся от неявных дубликатов в колонке `genre`.

In [14]:
#Просмотр уникальных названий жанров
df['genre'].sort_values().unique()

array(['acid', 'acoustic', 'action', 'adult', 'africa', 'afrikaans',
       'alternative', 'alternativepunk', 'ambient', 'americana',
       'animated', 'anime', 'arabesk', 'arabic', 'arena',
       'argentinetango', 'art', 'audiobook', 'author', 'avantgarde',
       'axé', 'baile', 'balkan', 'beats', 'bigroom', 'black', 'bluegrass',
       'blues', 'bollywood', 'bossa', 'brazilian', 'breakbeat', 'breaks',
       'broadway', 'cantautori', 'cantopop', 'canzone', 'caribbean',
       'caucasian', 'celtic', 'chamber', 'chanson', 'children', 'chill',
       'chinese', 'choral', 'christian', 'christmas', 'classical',
       'classicmetal', 'club', 'colombian', 'comedy', 'conjazz',
       'contemporary', 'country', 'cuban', 'dance', 'dancehall',
       'dancepop', 'dark', 'death', 'deep', 'deutschrock', 'deutschspr',
       'dirty', 'disco', 'dnb', 'documentary', 'downbeat', 'downtempo',
       'drum', 'dub', 'dubstep', 'eastern', 'easy', 'electronic',
       'electropop', 'emo', 'entehno', '

В списке названий жанров есть неявные дубликаты названия hiphop (hip, hiphop, hop). Напишу функцию для их замены. 

In [15]:
#функция для замены неявных дубликатов
def replace_wrong_genres(wrong_genres,correct_genre):
    for wrong_genre in wrong_genres:
        df['genre'] = df['genre'].replace(wrong_genre, correct_genre)    

In [16]:
#устраню неявные дубликаты
duplicates = ['hip', 'hop', 'hip-hop']
correct = 'hiphop'
replace_wrong_genres(duplicates, correct)

In [17]:
print(df['genre'].sort_values().unique()) #проверка

['acid' 'acoustic' 'action' 'adult' 'africa' 'afrikaans' 'alternative'
 'alternativepunk' 'ambient' 'americana' 'animated' 'anime' 'arabesk'
 'arabic' 'arena' 'argentinetango' 'art' 'audiobook' 'author' 'avantgarde'
 'axé' 'baile' 'balkan' 'beats' 'bigroom' 'black' 'bluegrass' 'blues'
 'bollywood' 'bossa' 'brazilian' 'breakbeat' 'breaks' 'broadway'
 'cantautori' 'cantopop' 'canzone' 'caribbean' 'caucasian' 'celtic'
 'chamber' 'chanson' 'children' 'chill' 'chinese' 'choral' 'christian'
 'christmas' 'classical' 'classicmetal' 'club' 'colombian' 'comedy'
 'conjazz' 'contemporary' 'country' 'cuban' 'dance' 'dancehall' 'dancepop'
 'dark' 'death' 'deep' 'deutschrock' 'deutschspr' 'dirty' 'disco' 'dnb'
 'documentary' 'downbeat' 'downtempo' 'drum' 'dub' 'dubstep' 'eastern'
 'easy' 'electronic' 'electropop' 'emo' 'entehno' 'epicmetal' 'estrada'
 'ethnic' 'eurofolk' 'european' 'experimental' 'extrememetal' 'fado'
 'fairytail' 'film' 'fitness' 'flamenco' 'folk' 'folklore' 'folkmetal'
 'folkrock' 

**Выводы**

Выполнена предобработка данных, было обнаружено три проблемы в данных:

- нарушения в стиле заголовков,
- пропущенные значения,
- дубликаты — явные и неявные. 

Можно приступать к проверке гипотез. 

## Проверка гипотез

### Сравнение поведения пользователей двух столиц

Первая гипотеза: пользователи по-разному слушают музыку в Москве и Санкт-Петербурге. 
Проверим это предположение по данным о трёх днях недели — понедельнике, среде и пятнице.
Для этого:

In [18]:
#посчитаю число прослушиваний в каждом городе
df_group_city = df.groupby('city')['track'].count()
display(df_group_city)


city
Moscow              42741
Saint-Petersburg    18512
Name: track, dtype: int64

В Москве прослушиваний больше, чем в Петербурге, но и пользователей в Москве больше.

In [19]:
#посчитаю число прослушиваний в каждый из трёх дней
df_group_day = df.groupby('day')['track'].count()
display(df_group_day)

day
Friday       21840
Monday       21354
Wednesday    18059
Name: track, dtype: int64

В среднем пользователи из двух городов менее активны по средам. Но необходимо посмотреть города по отдельности.

In [20]:
#функция для подсчета прослушивания для заданного дня и города.
def number_tracks(day, city): 
    track_list = df[df['day'] == day]
    track_list = track_list[track_list['city'] == city]
    track_list_count = track_list['user_id'].count()
    return track_list_count

In [21]:
number_tracks('Monday', 'Moscow') # количество прослушиваний в Москве по понедельникам

15740

In [22]:
number_tracks('Monday', 'Saint-Petersburg')# количество прослушиваний в Санкт-Петербурге по понедельникам

5614

In [23]:
number_tracks('Wednesday', 'Moscow')# количество прослушиваний в Москве по средам

11056

In [24]:
number_tracks('Wednesday', 'Saint-Petersburg')# количество прослушиваний в Санкт-Петербурге по средам

7003

In [25]:
number_tracks('Friday', 'Moscow')# количество прослушиваний в Москве по пятницам

15945

In [26]:
number_tracks('Friday', 'Saint-Petersburg')# количество прослушиваний в Санкт-Петербурге по пятницам

5895

In [27]:
#создам таблицу по полученным данным
data = [
    ['Moscow', 15740, 11056, 15945],
    ['Saint-Petersburg', 5614, 7003, 5895]
]
columns = ['city', 'monday', 'wednesday', 'friday']
pd.DataFrame(data = data, columns = columns) # Таблица с результатами

,city,monday,wednesday,friday
0,Moscow,15740,11056,15945
1,Saint-Petersburg,5614,7003,5895


**Выводы**

Данные показывают разницу поведения пользователей:

- В Москве больше прослушиваний в понедельник и пятницу.
- В Петербурге, наоборот, больше всего прослушаваний по средам. Активность в понедельник и пятницу здесь почти в равной мере уступает среде.

Значит, можно предположить, что гипотеза верна. Пользователи по-разному слушают музыку в Москве и Санкт-Петербурге.

### Музыка в начале и в конце недели

Вторая гипотеза: утром в понедельник в Москве преобладают одни жанры, а в Петербурге — другие. Так же и вечером пятницы преобладают разные жанры — в зависимости от города.

In [28]:
# получение таблицы moscow_general из тех строк таблицы df, для которых значение в столбце 'city' равно 'Moscow'
moscow_general = df[df['city'] == 'Moscow']

In [29]:
# получение таблицы spb_general из тех строк таблицы df, для которых значение в столбце 'city' равно 'Saint-Petersburg'
spb_general = df[df['city'] == 'Saint-Petersburg']

In [30]:
#функция для получения информации о топ-10 жанров тех треков, которые прослушивали в указанный день, в промежутке между двумя отметками времени.
def genre_weekday(table, day, time1, time2):
    genre_df = table[table['day'] == day]
    genre_df = genre_df[genre_df['time_start'] > time1]
    genre_df = genre_df[genre_df['time_start'] < time2]  

    genre_df_count = genre_df.groupby('genre')['track'].count()

    genre_df_sorted = genre_df_count.sort_values(ascending = False)
    return genre_df_sorted.head(10)

Cравним результаты функции `genre_weekday()` для Москвы и Санкт-Петербурга в понедельник утром (с 7:00 до 11:00) и в пятницу вечером (с 17:00 до 23:00):

In [31]:
#вызов функции для утра понедельника в Москве
genre_weekday(moscow_general, 'Monday', '07:00', '11:00')

genre
pop            781
dance          549
electronic     480
rock           474
hiphop         286
ruspop         186
world          181
rusrap         175
alternative    164
unknown        161
Name: track, dtype: int64

In [32]:
#вызов функции для утра понедельника в Петербурге
genre_weekday(spb_general, 'Monday', '07:00', '11:00')

genre
pop            218
dance          182
rock           162
electronic     147
hiphop          80
ruspop          64
alternative     58
rusrap          55
jazz            44
classical       40
Name: track, dtype: int64

In [33]:
#вызов функции для вечера пятницы в Москве
genre_weekday(moscow_general, 'Friday', '17:00', '23:00')

genre
pop            713
rock           517
dance          495
electronic     482
hiphop         273
world          208
ruspop         170
alternative    163
classical      163
rusrap         142
Name: track, dtype: int64

In [34]:
# вызов функции для вечера пятницы в Петербурге
genre_weekday(spb_general, 'Friday', '17:00', '23:00')

genre
pop            256
electronic     216
rock           216
dance          210
hiphop          97
alternative     63
jazz            61
classical       60
rusrap          59
world           54
Name: track, dtype: int64

**Выводы**

Если сравнить топ-10 жанров в понедельник утром, можно сделать такие выводы:

1. В Москве и Петербурге слушают похожую музыку. Отличие — в московский рейтинг вошёл жанр “world”, а в петербургский — джаз и классика.

2. В Москве пропущенных значений оказалось так много, что значение `'unknown'` заняло десятое место среди самых популярных жанров. Значит, пропущенные значения занимают существенную долю в данных и влияют на качество исследования.

Вечер пятницы не меняет эту картину. Некоторые жанры поднимаются немного выше, другие спускаются, но в целом топ-10 остаётся тем же самым.

Таким образом, вторая гипотеза подтвердилась лишь частично:
* Пользователи слушают похожую музыку в начале недели и в конце.
* Разница между Москвой и Петербургом не слишком выражена. В Москве чаще слушают русскую популярную музыку, в Петербурге — джаз.

Однако пропуски в данных ставят под сомнение этот результат. В Москве их так много, что рейтинг топ-10 мог бы выглядеть иначе, если бы не утерянные  данные о жанрах.

### Жанровые предпочтения в Москве и Петербурге

Третья гипотеза: Петербург — столица рэпа, музыку этого жанра там слушают чаще, чем в Москве.  А Москва — город контрастов, в котором, тем не менее, преобладает поп-музыка.

In [35]:
#сгруппирую таблицу moscow_general по жанру и посчитаю просушивания треков
moscow_genres = moscow_general.groupby('genre')['genre'].count().sort_values(ascending = False)

In [36]:
moscow_genres.head(10)

genre
pop            5892
dance          4435
rock           3965
electronic     3786
hiphop         2096
classical      1616
world          1432
alternative    1379
ruspop         1372
rusrap         1161
Name: genre, dtype: int64

In [37]:
#сгруппирую таблицу spb_general по жанру и посчитаю просушивания треков
spb_genres = spb_general.groupby('genre')['genre'].count().sort_values(ascending = False)

In [38]:
spb_genres.head(10)

genre
pop            2431
dance          1932
rock           1879
electronic     1736
hiphop          960
alternative     649
classical       646
rusrap          564
ruspop          538
world           515
Name: genre, dtype: int64

**Вывод**

Гипотеза частично подтвердилась:
* Поп-музыка — самый популярный жанр в Москве, как и предполагала гипотеза. Более того, в топ-10 жанров встречается близкий жанр — русская популярная музыка.
* Рэп одинаково популярен в Москве и Петербурге. 


## Итоги исследования

Проверены три гипотезы и установлена:

1. День недели по-разному влияет на активность пользователей в Москве и Петербурге. 

Первая гипотеза полностью подтвердилась.

2. Музыкальные предпочтения не сильно меняются в течение недели — будь то Москва или Петербург. Небольшие различия заметны в начале недели, по понедельникам:
* в Москве слушают музыку жанра “world”,
* в Петербурге — джаз и классику.

Таким образом, вторая гипотеза подтвердилась лишь отчасти. Этот результат мог оказаться иным, если бы не пропуски в данных.

3. Во вкусах пользователей Москвы и Петербурга больше общего чем различий. Предпочтения жанров в Петербурге напоминают московские.

Третья гипотеза не подтвердилась. Если различия в предпочтениях и существуют, на основной массе пользователей они незаметны.